# Sustainable Capitol Hill 2024 Q1 Meeting

In [ ]:
import numpy as np
import pandas as pd

members = pd.read_pickle('data/output/members.pkl')

def to_month(timestamp_series):
    return timestamp_series.apply(lambda t: pd.Timestamp(t.year, t.month, 1))

member_dates = pd.DataFrame(
    {
        'Online Sign-Up Count': members['Created'].groupby(to_month(members['Created'])).size().asfreq('MS',fill_value=0),
        'In Person Activation Count ': members['First Membership Started'].groupby(to_month(members['First Membership Started'])).size().asfreq('MS',fill_value=0)
    },
    index=pd.date_range(freq='MS', start=pd.Timestamp('2018-01-01'), end=pd.Timestamp('2023-12-31'))
)

graph = member_dates.plot(
    title='Online Sign-ups And In-Person Activations Per Month',
    figsize=(10, 6),
    ylabel='Count',
    ylim=(0,140),
)

In [ ]:
import numpy as np
import pandas as pd

members = pd.read_pickle('data/output/members.pkl')

def to_year(timestamp_series):
    return timestamp_series.apply(lambda t: None if pd.isna(t.year) else int(t.year))

members['Created Year'] = to_year(members['Created'])
members['First Membership Started Year'] = to_year(members['First Membership Started'])

recent_years = [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
members = members[members['Created Year'].isin(recent_years) & (members['First Membership Started Year'].isna() | members['First Membership Started Year'].isin(recent_years))]

member_dates = pd.DataFrame(
    {
        'Online Sign-Up Count': members['Created Year'].groupby(members['Created Year']).size(),
        'In Person Activation Count ': members['First Membership Started Year'].groupby(members['First Membership Started Year']).size()
    }
)

graph = member_dates.plot(
    kind='bar',
    grid=True,
    title='Online Sign-ups And In-Person Activations Per Year',
    figsize=(10, 6),
    ylabel='Count',
    ylim=(0,1400),
)

In [ ]:
import pandas as pd
from datetime import date

checkouts = pd.read_pickle('data/output/checkouts.pkl')
checkouts['Month'] = pd.to_datetime(checkouts['Checked Out'].map(lambda c: date(c.year, c.month, 1)))
checkouts['Year'] = checkouts['Checked Out'].map(lambda c: c.year)
# Exclude 2015 (tool library not fully open) and current year if it is only a month or two in.
recent_years = checkouts[checkouts['Year'].isin([2018, 2019, 2020, 2021, 2022, 2023])]
by_month = recent_years.groupby('Month').size().asfreq('MS', fill_value=0)

# Save to a variable so the __str__ doesn't get displayed
graph = by_month.plot(figsize=(10, 6), title='Checkouts Per Month', ylabel='Count', ylim=0)

In [ ]:
# Checkout totals per year
import pandas as pd
from datetime import date

checkouts = pd.read_pickle('data/output/checkouts.pkl')
checkouts['Year'] = checkouts['Checked Out'].map(lambda c: c.year)
# Exclude 2015 (tool library not fully open) and current year if it is only a month or two in.
recent_years = checkouts[checkouts['Year'].isin([2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023])]

# Save to a variable so the __str__ doesn't get displayed
graph = recent_years.groupby('Year').size().plot(kind='bar', figsize=(10, 6), title='Checkouts Per Year', grid=True, ylabel='Count', ylim=0)

In [ ]:
from datetime import date
import pandas as pd

loans = pd.read_pickle('data/output/loans.pkl')
loans['Length'] = loans['Checked In'].fillna(pd.to_datetime(date.today())) - loans['Checked Out']
# Grouping by the "Year" it was checked out artificially reduces the most recent years loan length.
# Previous years get all unreturned items counted for the whole year. In the current year there isn't
# enough time for those to add up yet.
loans['Year'] = loans['Checked Out'].map(lambda c: c.year)
recent_years = loans[loans['Year'].isin([2018, 2019, 2020, 2021, 2022, 2023, 2024])]

by_year = pd.DataFrame({'Days': recent_years['Length'].map(lambda l: l.days), 'Year': recent_years['Year']})
graph = by_year.pivot(columns='Year', values='Days').plot.box(
    # "fliers" in plt parlance are outliers.  We don't show them here because there are so many, they make the
    # rest of the plot much harder to read.  The gist is that a few loans are just never returned.
    showfliers=False,
    figsize=(10, 6),
    title='Item Loan Length',
    ylabel='Days Loaned',
)

## Member Stories

These are some of the projects I've heard people mention during my shifts:

> I am refinishing my industrial loft

> I added a hole to my belt loop while I was in the back

> I just moved to Seattle, and want to hang a shelf but don't have any tools.

> Someone just broke up with me 5 minute ago and I need to agressively cut wood

> I'd like to shorten this bit of metal shelving.

> I'm demolishing my kitchen.

## Income

### \\$19k revenue - \\$5k expenses = \\$14k net income

## Revenue

The Capitol Hill Tool Library gets revenue from membership donations, late fees, and donations directly to Sustainable Capitol Hill. The first two kinds of fees are recorded in MyTurn, and direct donations in the Sustainable Capitol Hill BECU account.


### Late Fees

In [ ]:
import pandas as pd

transactions = pd.read_pickle('data/output/transactions.pkl')
transactions['Year'] = transactions['Date'].apply(lambda d: d.year)

payments = transactions[transactions['Payment method'].notna()]
payments = payments[payments['Year'].isin([2018, 2019, 2020, 2021, 2022, 2023])]

late_fees = pd.DataFrame(payments[payments['Item Type'] == 'Late Fee'])
membership_fees = pd.DataFrame(payments[payments['Item Type'] == 'Membership Fee'])


graph = late_fees.groupby('Year').sum(numeric_only=True)[['Payment Amount', 'Discount']].plot.bar(
    stacked=True, title='Late Fees', grid=True, ylabel="Dollars", xlabel="")
# TODO: MyTurn records over-payment donations as negative amounts. Need to think
# more about what kind of stats I'd want to record for late fees.

### Membership Donations

In [ ]:
membership_fees['Year'] = membership_fees['Date'].apply(lambda d: d.year)
by_year = membership_fees.groupby('Year')
graph = by_year.sum(numeric_only=True)[['Payment Amount']].plot.bar(
    stacked=True, title='Membership Donations', grid=True, ylabel="Dollars", xlabel="")

graph = membership_fees[membership_fees['Payment Amount'] >= 0].pivot(columns='Year', values='Payment Amount').plot.box(
    ylabel="Dollars",
    showfliers=False,
)


### Direct Donations

People donate directly to Sustainable Capitol Hill and the tool library, both in-person and online.  We received an additional ~$7k in direct donations.

### Expenses

Our primary expenses are rent (\~1.5k), insurance (\~2k), web hosting (\~.5k), and money spent repairing tools.

#### About This Report
Code and instructions for playing with the Capitol Hill Tool Library data yourself are available at https://github.com/mshenfield/chtl-data-pipeline. Ask [Max Shenfield](https://app.slack.com/client/TAAD5LKJ4/D02BHR5J7J6/user_profile/U02BB35KMRU) for access.  The project is built using [Python](https://python.org), [Pandas](https://pandas.pydata.org/), and [Jupyter Notebook](https://jupyter.org/).